In [1]:
import findspark
findspark.add_jars('/app/postgresql-42.1.4.jar')
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
    .appName("argentinaETL")
    .config("spark.driver.memory", "512m")
    .config("spark.driver.cores", "1")
    .config("spark.executor.memory", "512m")
    .config("spark.executor.cores", "1")
    .config("spark.sql.shuffle.partitions", "2")
    .getOrCreate()
)

In [3]:
from pyspark.sql.functions import *;
from datetime import datetime, date

In [4]:
from pyspark.sql.types import *

schema = (
StructType().
    add("ID_Provincia", IntegerType()).
    add("Provincia", StringType()).
    add("ID_Departamento", IntegerType()).
    add("Departamento", StringType()).
    add("Id_Cultivo", IntegerType()).
    add("Cultivo", StringType()).
    add("ID_Campania", IntegerType()).
    add("Campania", StringType()).
    add("Ha_Sembrada", DecimalType()).
    add("Ha_Cosechada", DecimalType()).
    add("Produccion_Tn", DecimalType()).
    add("Rendimiento_KgxHa", DecimalType())
)

In [5]:
df = (
    spark.read.
    option("delimiter", ';').
    csv("/dataset/argentina-datos/Estimaciones/Estimaciones.csv",
        schema=schema,
        header=True,
        ignoreLeadingWhiteSpace=True,
        ignoreTrailingWhiteSpace=True,
        nullValue='NA')
     )

In [6]:
df = df.withColumnRenamed("ID_Provincia", "id_provincia") \
.withColumnRenamed("Provincia", "provincia") \
.withColumnRenamed("ID_Departamento", "id_departamento") \
.withColumnRenamed("Departamento", "departamento") \
.withColumnRenamed("ID_Cultivo", "id_cultivo") \
.withColumnRenamed("Cultivo", "cultivo") \
.withColumnRenamed("ID_Campania", "id_campania") \
.withColumnRenamed("Campania", "campania") \
.withColumnRenamed("Ha_Sembrada", "ha_sembrada") \
.withColumnRenamed("Ha_Cosechada", "ha_cosechada") \
.withColumnRenamed("Produccion_Tn", "produccion_tn") \
.withColumnRenamed("Rendimiento_KgxHa", "rendimiento_kgha")



In [12]:
df.describe(["ha_sembrada"]).show();
df.describe(["ha_cosechada"]).show();
df.describe(["produccion_tn"]).show();
df.describe(["rendimiento_kgha"]).show();

+-------+-----------------+
|summary|      ha_sembrada|
+-------+-----------------+
|  count|           148162|
|   mean|       11985.8442|
| stddev|35267.86689359856|
|    min|                0|
|    max|           894613|
+-------+-----------------+

+-------+----------------+
|summary|    ha_cosechada|
+-------+----------------+
|  count|          148162|
|   mean|      10488.4912|
| stddev|33845.4010526086|
|    min|               0|
|    max|          883613|
+-------+----------------+

+-------+------------------+
|summary|     produccion_tn|
+-------+------------------+
|  count|            148162|
|   mean|        32573.3768|
| stddev|126465.23307389554|
|    min|                 0|
|    max|           8620000|
+-------+------------------+

+-------+-----------------+
|summary| rendimiento_kgha|
+-------+-----------------+
|  count|           148162|
|   mean|        3525.9199|
| stddev|5670.321724013817|
|    min|                0|
|    max|           300000|
+-------+--------

In [13]:
for col in df.columns:
    df.describe([col]).show()

+-------+------------------+
|summary|      id_provincia|
+-------+------------------+
|  count|            148162|
|   mean| 34.04365491826514|
| stddev|29.346447617326437|
|    min|                 6|
|    max|                94|
+-------+------------------+

+-------+------------+
|summary|   provincia|
+-------+------------+
|  count|      148162|
|   mean|        null|
| stddev|        null|
|    min|BUENOS AIRES|
|    max|     TUCUMAN|
+-------+------------+

+-------+------------------+
|summary|   id_departamento|
+-------+------------------+
|  count|            148162|
|   mean|  191.189711261997|
| stddev|228.58633087604437|
|    min|                 0|
|    max|               882|
+-------+------------------+

+-------+------------+
|summary|departamento|
+-------+------------+
|  count|      148162|
|   mean|        null|
| stddev|        null|
|    min|   1 DE MAYO|
|    max|       ZONDA|
+-------+------------+

+-------+------------------+
|summary|        id_cultivo|
+-

In [14]:
df.select('departamento').distinct().show()

+--------------------+
|        departamento|
+--------------------+
|         SIN DEFINIR|
|          TAFI VIEJO|
|         YERBA BUENA|
|          25 DE MAYO|
|       ADOLFO ALSINA|
|ADOLFO GONZALES C...|
|             ALBERTI|
|                AZUL|
|            BARADERO|
|       BENITO JUAREZ|
|             CAMPANA|
|            CANUELAS|
|   CAPITAN SARMIENTO|
|     CARMEN DE ARECO|
|           CHASCOMUS|
|               COLON|
|     CORONEL DORREGO|
|    CORONEL PRINGLES|
|      CORONEL SUAREZ|
|            DAIREAUX|
+--------------------+
only showing top 20 rows



In [15]:
from pyspark.sql.functions import when, lit

df = df.withColumn('cultivo', when(df['cultivo']=="Ca�a de az�car",lit("Cania de azucar")).otherwise(df['Cultivo']));
df = df.withColumn('cultivo', when(df['cultivo']=="T�", lit("Te")).otherwise(df['Cultivo']));
df = df.withColumn('cultivo', when(df['cultivo']=="Algod�n", lit("Algodon")).otherwise(df['Cultivo']));
df = df.withColumn('cultivo', when(df['cultivo']=="C�rtamo", lit("Cartamo")).otherwise(df['Cultivo']));
df = df.withColumn('cultivo', when(df['cultivo']=="Man�", lit("Mani")).otherwise(df['Cultivo']));
df = df.withColumn('cultivo', when(df['cultivo']=="Ma�z", lit("Maiz")).otherwise(df['Cultivo']));


In [16]:
df.select('cultivo').distinct().show()

+----------------+
|         cultivo|
+----------------+
|        Soja 1ra|
|      Soja total|
|              Te|
|             Ajo|
|           Arroz|
|          Arveja|
| Cania de azucar|
|Cebada cervecera|
|Cebada forrajera|
|   Cebolla total|
|        Garbanzo|
|         Girasol|
|          Jojoba|
|         Lenteja|
|            Maiz|
|            Mani|
|      Papa total|
|        Soja 2da|
|           Sorgo|
|   Trigo candeal|
+----------------+
only showing top 20 rows



In [17]:
for col in df.columns:
    print(col, "\t", "Nulls: ", df.filter(df[col]=="NA").count())

id_provincia 	 Nulls:  0
provincia 	 Nulls:  0
id_departamento 	 Nulls:  0
departamento 	 Nulls:  0
id_cultivo 	 Nulls:  0
cultivo 	 Nulls:  0
id_campania 	 Nulls:  0
campania 	 Nulls:  0
ha_sembrada 	 Nulls:  0
ha_cosechada 	 Nulls:  0
produccion_tn 	 Nulls:  0
rendimiento_kgha 	 Nulls:  0


In [18]:
df = df.withColumn('anio', split(df['campania'], '/').getItem(0));

In [19]:
df = df.withColumn('pre', lit("-01-01"))

In [20]:
df = df.withColumn('fecha',to_date(concat(df['anio'], df['pre']), 'yyyy-mm-dd'))

In [21]:
df = df.drop('anio', 'pre')

In [22]:
df.printSchema()

root
 |-- id_provincia: integer (nullable = true)
 |-- provincia: string (nullable = true)
 |-- id_departamento: integer (nullable = true)
 |-- departamento: string (nullable = true)
 |-- id_cultivo: integer (nullable = true)
 |-- cultivo: string (nullable = true)
 |-- id_campania: integer (nullable = true)
 |-- campania: string (nullable = true)
 |-- ha_sembrada: decimal(10,0) (nullable = true)
 |-- ha_cosechada: decimal(10,0) (nullable = true)
 |-- produccion_tn: decimal(10,0) (nullable = true)
 |-- rendimiento_kgha: decimal(10,0) (nullable = true)
 |-- fecha: date (nullable = true)



In [23]:
df.show(2)

+------------+------------+---------------+------------+----------+-------+-----------+--------+-----------+------------+-------------+----------------+----------+
|id_provincia|   provincia|id_departamento|departamento|id_cultivo|cultivo|id_campania|campania|ha_sembrada|ha_cosechada|produccion_tn|rendimiento_kgha|     fecha|
+------------+------------+---------------+------------+----------+-------+-----------+--------+-----------+------------+-------------+----------------+----------+
|           6|BUENOS AIRES|            854|  25 DE MAYO|         1|    Ajo|          1| 1969/70|          3|           3|           10|            3333|1969-01-01|
|           6|BUENOS AIRES|            854|  25 DE MAYO|         1|    Ajo|          2| 1970/71|          1|           1|            3|            3000|1970-01-01|
+------------+------------+---------------+------------+----------+-------+-----------+--------+-----------+------------+-------------+----------------+----------+
only showing top

In [ ]:
### Write to postgres

In [24]:
df \
    .write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres/agricultura") \
    .option("dbtable", "agricultura.argentina_datos") \
    .option("user", "agricultura") \
    .option("password", "p4ssW0rdP4r4Agr1cultur4") \
    .option("driver", "org.postgresql.Driver") \
    .mode('overwrite') \
    .save()

In [25]:
spark.stop()